In [2]:
import codecs
import json
import logging
import os
import sys
import operator
from collections import defaultdict
from tokenizer import twokenize
from twitter.Tweet import Tweet
from util import locations, ngrams
from processing import locationsInTweets
import stats_enriched_tweets

In [3]:
# load cities and countries
countries = locations.Countries.loadFromFile()
euroCountries = locations.Countries.filterEuropeanCountries(countries)
cities = locations.Cities.loadFromFile()
euroCities = locations.Cities.filterEuropeanCities(cities)
ccDict = locations.Countries.countryCodeDict(countries)

current dir:  /Users/cris/Documents/workspace/TwitterCrisis
resources/countryInfo.txt
All countries:  166
European countries:  55
All cities:  6442
European cities:  1962


In [36]:
print euroCities["derry"]
print euroCountries["ireland"]

(u'derry', u'Derry', -7.30934, 54.9981, u'GB', 83652, u'Europe/London')
(u'ireland', u'Dublin', u'4622917', u'EU', u'IE')


In [4]:
# read enriched tweet
def tweetIter(inputfile):
    for line in codecs.open(inputfile, "r", "utf-8"):
        try:
            tweet = json.loads(line)
        except:
            print "Couldn't parse tweet: ", line[:200]
        yield tweet

In [5]:
def smallerCity(tweet, euroCities):
    if len(tweet["user_ex_location"]) > 1:
        # same country different cities
        locOrderedByPop = dict()
        for loc in tweet["user_ex_location"]:
            population = euroCities[loc][5]
            locOrderedByPop[loc] = population
        sorted_x = sorted(locOrderedByPop.items(), key=operator.itemgetter(1))

        return sorted_x[0][0]
    else:
        return tweet["user_ex_location"][0]

In [32]:
def placeConsistecy(tweet, cityDict, countryDict):
    user_loc_city = smallerCity(tweet, euroCities)
    user_loc_country_cc = tweet["user_ex_location_c"]
    # comparison by country code
    try:
        city_cc = cityDict[user_loc_city][4]
    except:
        print "city", user_loc_city, cityDict[user_loc_city]
    
    if city_cc != user_loc_country_cc:
        print city_cc, user_loc_city, user_loc_country_cc
        print tweet["user_location"]

In [37]:
inputFilename = "/Users/cris/Final2/refugees_all.json"
tweetsAsDict = tweetIter(inputFilename)

i = 0
for tweet in tweetsAsDict:
    i+=1
    if tweet["user_ex_location_c"] and len(tweet["user_ex_location"]) > 0:
        placeConsistecy(tweet, cities, countries)
        
    #break

CZ prague CH
London, Switzerland, Prague
GB newtownabbey IE
Newtownabbey,Northern Ireland
CZ prague CH
London, Switzerland, Prague
CZ prague CH
London, Switzerland, Prague
NO oslo MT
Malta/London/Oslo 
FR strasbourg GB
UK, Brussels & Strasbourg
ES barcelona AD
London, Barcelona, Andorra
GB derry IE
Derry, Ireland
GB derry IE
Derry, Ireland
CZ prague CH
London, Switzerland, Prague
CZ prague CH
London, Switzerland, Prague
CZ prague CH
London, Switzerland, Prague
ES barcelona AD
London, Barcelona, Andorra
GB belfast IE
Belfast Northern Ireland 
CZ prague CH
London, Switzerland, Prague
CZ prague CH
London, Switzerland, Prague
GB belfast IE
Belfast, Northern Ireland
CZ prague CH
London, Switzerland, Prague
CZ prague CH
London, Switzerland, Prague
GB derry IE
Derry, Ireland
GB derry IE
Derry, Ireland
CZ prague CH
London, Switzerland, Prague
DE herne GB
Herne Bay, England
CZ prague CH
London, Switzerland, Prague
GB luton GR
Luton, England/Patras, Greece
TR izmir NL
Izmir, Turkey / Netherlands